For more details on this tutorial's setup and process, see `00_Introduction.ipynb`.

# Stage 4: Regular Batch Inference

In Stage 3: Deploy the Model in Wallaroo, the housing model created and tested in Stage 2: Training Process Automation Setup was uploaded to a Wallaroo instance and added to the pipeline `housing-pipe` in the workspace `housepricing`.  This pipeline can be deployed at any point and time and used with new inferences.

For the purposes of this demo, let's say that every month we find the newly entered and still-unsold houses and predict their sale price.

The predictions are entered into a staging table, for further inspection before being joined to the primary housing data table.

We show this as a notebook, but this can also be scripted and scheduled, using CRON or some other process.

## Resources

The following resources are used as part of this tutorial:

* **data**
  * `data/seattle_housing_col_description.txt`: Describes the columns used as part data analysis.
  * `data/seattle_housing.csv`: Sample data of the Seattle, Washington housing market between 2014 and 2015.
* **code**
  * `postprocess.py`: Formats the data after inference by the model is complete.
  * `preprocess.py`: Formats the incoming data for the model.
  * `simdb.py`: A simulated database to demonstrate sending and receiving queries.
  * `wallaroo_client.py`: Additional methods used with the Wallaroo instance to create workspaces, etc.
* **models**
  * `housing_model_xgb.onnx`: Model created in Stage 2: Training Process Automation Setup.

## Steps

This process will use the following steps:

* [Connect to Wallaroo](#connect-to-wallaroo): Connect to the Wallaroo instance and the `housepricing` workspace.
* [Deploy the Pipeline](#deploy-the-pipeline): Deploy the pipeline to prepare it to run inferences.
* [Read In New House Listings](#read-in-new-house-listings): Read in the previous month's house listings and submit them to the pipeline for inference.
* [Send Predictions to Results Staging Table](#send-predictions-to-results-staging-table): Add the inference results to the results staging table.

### Connect to Wallaroo

Connect to the Wallaroo instance and set the `housepricing` workspace as the current workspace.

In [1]:
import json
import pickle
import wallaroo
import pandas as pd
import numpy as np
import pyarrow as pa

import simdb # module for the purpose of this demo to simulate pulling data from a database

from wallaroo_client import get_workspace

import os
# Only set the below to make the OS environment ARROW_ENABLED to TRUE.  Otherwise, leave as is.
os.environ["ARROW_ENABLED"]="True"

# used to display dataframe information without truncating
from IPython.display import display
pd.set_option('display.max_colwidth', None)

import preprocess

import postprocess

In [2]:
wallaroo.__version__

'2023.2.0+05382cff8'

In [3]:
# Login through local Wallaroo instance

wl = wallaroo.Client()

# SSO login through keycloak

# wallarooPrefix = "YOUR PREFIX"
# wallarooSuffix = "YOUR PREFIX"

# wallarooPrefix = "doc-test"
# wallarooSuffix = "wallaroocommunity.ninja"

# wl = wallaroo.Client(api_endpoint=f"https://{wallarooPrefix}.api.{wallarooSuffix}", 
#                     auth_endpoint=f"https://{wallarooPrefix}.keycloak.{wallarooSuffix}", 
#                     auth_type="sso")

In [4]:
def get_workspace(name):
    workspace = None
    for ws in wl.list_workspaces():
        if ws.name() == name:
            workspace= ws
    if(workspace == None):
        workspace = wl.create_workspace(name)
    return workspace

def get_pipeline(name):
    try:
        pipeline = wl.pipelines_by_name(name)[0]
    except EntityNotFoundError:
        pipeline = wl.build_pipeline(name)
    return pipeline

In [5]:
workspace_name = 'housepricing'
model_name = "housepricemodel"
model_file = "./housing_model_xgb.onnx"
pipeline_name = "housing-pipe"

In [6]:
new_workspace = get_workspace(workspace_name)
_ = wl.set_current_workspace(new_workspace)

### Deploy the Pipeline

Deploy the `housing-pipe` workspace established in Stage 3: Deploy the Model in Wallaroo (`03_deploy_model.ipynb`).

In [7]:
pipeline = get_pipeline(pipeline_name)
pipeline.deploy()

Waiting for deployment - this will take up to 45s ............ ok


name,housing-pipe
created,2023-05-01 18:43:09.144571+00:00
last_updated,2023-05-01 18:57:02.686956+00:00
deployed,True
tags,
versions,"fe149f8f-e278-49df-95f3-b385e9d361b6, 9ea1be58-bc23-45db-a383-aa87eb2d3d01, 16de3f12-2e15-47a7-bf10-59976831df21, c28f1bfc-786c-495f-aead-c653429e0d3a, 42bbc395-c30f-4d04-976b-ebb2ad4bd598"
steps,preprocess


### Read In New House Listings

From the data store, load the previous month's house listing, prepare it as a DataFrame, then submit it for inferencing.

In [8]:
conn = simdb.simulate_db_connection()

# create the query
query = f"select * from {simdb.tablename} where date > DATE(DATE(), '-1 month') AND sale_price is NULL"
print(query)

# read in the data
newbatch = pd.read_sql_query(query, conn)
newbatch.shape

select * from house_listings where date > DATE(DATE(), '-1 month') AND sale_price is NULL


(964, 22)

In [9]:
# if arrowEnabled is True:
#     query = {'query': newbatch.to_json()}
#     result = pipeline.infer(query)
#     #display(result)
#     predicted_prices = result[0]['prediction']
# else:
#     query = {'query': newbatch.to_json()}
#     result = pipeline.infer(query)[0]
#     predicted_prices = result.data()[0]
# len(predicted_prices)

query = {'query': newbatch.to_json()}
result = pipeline.infer(query)
#display(result)
predicted_prices = result[0]['prediction']
display(predicted_prices[0:5])

[508255.0, 500198.0, 539598.0, 270739.0, 346586.0]

In [14]:
display(result[0]['prediction'])

[508255.0,
 500198.0,
 539598.0,
 270739.0,
 346586.0,
 3067263.0,
 378917.0,
 496464.0,
 424550.0,
 1971057.0,
 343802.0,
 208110.0,
 252176.0,
 626480.0,
 351021.0,
 379524.0,
 1325326.0,
 281043.0,
 647002.0,
 529792.0,
 755094.0,
 258493.0,
 152646.0,
 401782.0,
 366496.0,
 445343.0,
 516452.0,
 192182.0,
 303202.0,
 207146.0,
 620057.0,
 412424.0,
 489603.0,
 360618.0,
 291413.0,
 232305.0,
 235307.0,
 553469.0,
 802369.0,
 343704.0,
 650089.0,
 418921.0,
 395079.0,
 376702.0,
 292946.0,
 192545.0,
 235490.0,
 577007.0,
 647420.0,
 309799.0,
 718814.0,
 2260575.0,
 1453942.0,
 328445.0,
 260539.0,
 444798.0,
 762827.0,
 189220.0,
 745876.0,
 640121.0,
 533739.0,
 642120.0,
 1208361.0,
 439301.0,
 556350.0,
 442150.0,
 668451.0,
 243662.0,
 371208.0,
 395673.0,
 460519.0,
 399616.0,
 298400.0,
 360618.0,
 291844.0,
 321080.0,
 224267.0,
 381861.0,
 278785.0,
 197249.0,
 306817.0,
 336451.0,
 322933.0,
 390226.0,
 905129.0,
 292168.0,
 363604.0,
 872705.0,
 315882.0,
 259630.0,
 613

### Send Predictions to Results Staging Table

Take the predicted prices based on the inference results so they can be joined into the `house_listings` table.

Once complete, undeploy the pipeline to return the resources back to the Kubernetes environment.

In [15]:
result_table = pd.DataFrame({
    'id': newbatch['id'],
    'saleprice_estimate': result[0]['prediction']
})

result_table.to_sql('results_table', conn, index=False, if_exists='append')

In [16]:
# Display the top of the table for confirmation
pd.read_sql_query("select * from results_table limit 5", conn)

,id,saleprice_estimate
0,9215400105,508255.0
1,1695900060,500198.0
2,9545240070,539598.0
3,1432900240,270739.0
4,1400300055,346586.0


In [17]:
conn.close()
pipeline.undeploy()

Waiting for undeployment - this will take up to 45s .................................... ok


name,housing-pipe
created,2023-05-01 18:43:09.144571+00:00
last_updated,2023-05-01 18:57:02.686956+00:00
deployed,False
tags,
versions,"fe149f8f-e278-49df-95f3-b385e9d361b6, 9ea1be58-bc23-45db-a383-aa87eb2d3d01, 16de3f12-2e15-47a7-bf10-59976831df21, c28f1bfc-786c-495f-aead-c653429e0d3a, 42bbc395-c30f-4d04-976b-ebb2ad4bd598"
steps,preprocess


From here, organizations can automate this process.  Other features could be used such as data analysis using Wallaroo assays, or other features such as shadow deployments to test champion and challenger models to find which models provide the best results.